# 🎨 ComfyUI Model Download Manager\n\n**Interactive model downloader with storage management**\n\n---\n\n## 📋 Features:\n\n- ✅ **Storage Selection** (Permanent/Temporary/Project)\n- ✅ **Model Categories** (Chroma, FLUX, SDXL, SD1.5)\n- ✅ **Space Management** (50GB Free Tier aware)\n- ✅ **Progress Tracking** with visual feedback\n- ✅ **Automatic Symlinks** to ComfyUI\n\n---

## 🔍 1. Check Storage Status

In [ ]:
import os\nimport shutil\nimport subprocess\nfrom pathlib import Path\nimport ipywidgets as widgets\nfrom IPython.display import display, HTML\n\n# Storage paths\nSTORAGE_PATH = Path('/storage/ComfyUI')\nTEMP_STORAGE = Path('/temp-storage/ComfyUI')\nPROJECT_PATH = Path('/comfyui-paperspace-notebook/ComfyUI')\n\ndef check_storage():\n    print(\"📊 STORAGE STATUS\")\n    print(\"================\")\n    \n    for name, path in [(\"Permanent\", \"/storage\"), (\"Temporary\", \"/temp-storage\"), (\"Project\", \"/comfyui-paperspace-notebook\")]:\n        try:\n            stat = shutil.disk_usage(path)\n            total_gb = stat.total / (1024**3)\n            used_gb = stat.used / (1024**3)\n            free_gb = stat.free / (1024**3)\n            percent = (stat.used / stat.total) * 100\n            \n            # Color coding\n            if percent > 90:\n                color = \"red\"\n            elif percent > 70:\n                color = \"orange\"\n            else:\n                color = \"green\"\n            \n            print(f\"\\n📁 {name} Storage ({path}):\")\n            print(f\"   Total: {total_gb:.1f}GB | Used: {used_gb:.1f}GB | Free: {free_gb:.1f}GB\")\n            print(f\"   <span style='color:{color}'>Usage: {percent:.1f}%</span>\")\n        except:\n            print(f\"\\n📁 {name} Storage: Not available\")\n\ncheck_storage()

## 🎯 2. Interactive Model Downloader

In [ ]:
# Interactive model selection\nimport json\nimport requests\nfrom concurrent.futures import ThreadPoolExecutor\nimport threading\n\nclass ModelDownloader:\n    def __init__(self):\n        self.storage_type = 'temp'\n        self.base_path = TEMP_STORAGE\n        self.selected_models = []\n        self.progress = widgets.IntProgress(min=0, max=100, description='Ready')\n        self.status = widgets.HTML(\"<b>Status:</b> Waiting for selection\")\n        \n    def download_file(self, url, dest_path, filename=None):\n        \"\"\"Download file with progress tracking\"\"\"\n        if not filename:\n            filename = os.path.basename(url)\n        \n        dest_path.mkdir(parents=True, exist_ok=True)\n        file_path = dest_path / filename\n        \n        # Skip if exists\n        if file_path.exists():\n            print(f\"✅ Already exists: {filename}\")\n            return True\n        \n        print(f\"⬇️ Downloading: {filename}\")\n        \n        try:\n            cmd = f\"wget -c -q --show-progress -O '{file_path}' '{url}'\"\n            result = subprocess.run(cmd, shell=True, capture_output=False, text=True)\n            if result.returncode == 0:\n                print(f\"✅ Downloaded: {filename}\")\n                return True\n            else:\n                print(f\"❌ Failed: {filename}\")\n                return False\n        except Exception as e:\n            print(f\"❌ Error downloading {filename}: {e}\")\n            return False\n\ndownloader = ModelDownloader()\n\n# Storage selection\nstorage_dropdown = widgets.Dropdown(\n    options=[\n        ('Temporary Storage (Session only)', 'temp'),\n        ('Permanent Storage (50GB limit)', 'perm'),\n        ('Project Directory', 'proj')\n    ],\n    value='temp',\n    description='Storage:',\n    style={'description_width': 'initial'}\n)\n\ndef on_storage_change(change):\n    if change['new'] == 'temp':\n        downloader.base_path = TEMP_STORAGE\n    elif change['new'] == 'perm':\n        downloader.base_path = STORAGE_PATH\n    else:\n        downloader.base_path = PROJECT_PATH\n    print(f\"📁 Storage set to: {downloader.base_path}\")\n\nstorage_dropdown.observe(on_storage_change, names='value')\n\n# Model selection checkboxes\nprint(\"\\n🎨 CHROMA MODELS:\")\nchroma_v48 = widgets.Checkbox(description='Chroma v48 (Latest)', value=False)\nchroma_v48_detail = widgets.Checkbox(description='Chroma v48 Detail', value=False)\nchroma_v29 = widgets.Checkbox(description='Chroma v29.5', value=False)\nchroma_vae = widgets.Checkbox(description='Chroma VAE (Required)', value=True, disabled=True)\nchroma_te = widgets.Checkbox(description='Text Encoder FP8', value=True)\n\nprint(\"\\n🔥 CHROMA LORAS:\")\nlora_realfine = widgets.Checkbox(description='RealFine LoRA', value=False)\nlora_turbo = widgets.Checkbox(description='Turbo LoRA', value=False)\nlora_hyper8 = widgets.Checkbox(description='Hyper 8-steps', value=False)\nlora_hyper16 = widgets.Checkbox(description='Hyper Turbo 16-steps', value=False)\n\nprint(\"\\n⚡ FLUX MODELS:\")\nflux_schnell = widgets.Checkbox(description='FLUX.1 Schnell', value=False)\nflux_dev = widgets.Checkbox(description='FLUX.1 Dev', value=False)\n\nprint(\"\\n🎯 SDXL MODELS:\")\nsdxl_base = widgets.Checkbox(description='SDXL Base 1.0', value=False)\nsdxl_refiner = widgets.Checkbox(description='SDXL Refiner', value=False)\nsdxl_vae = widgets.Checkbox(description='SDXL VAE', value=False)\n\n# Display widgets\ndisplay(storage_dropdown)\nprint(\"\\n📦 Select Models to Download:\")\ndisplay(widgets.VBox([\n    widgets.HTML(\"<b>Chroma Models:</b>\"),\n    chroma_v48, chroma_v48_detail, chroma_v29, chroma_vae, chroma_te,\n    widgets.HTML(\"<b>Chroma LoRAs:</b>\"),\n    lora_realfine, lora_turbo, lora_hyper8, lora_hyper16,\n    widgets.HTML(\"<b>FLUX Models:</b>\"),\n    flux_schnell, flux_dev,\n    widgets.HTML(\"<b>SDXL Models:</b>\"),\n    sdxl_base, sdxl_refiner, sdxl_vae\n]))\n\n# Download button\ndef download_selected(b):\n    downloads = []\n    \n    # Chroma models\n    if chroma_v48.value:\n        downloads.append((\"https://huggingface.co/lodestones/Chroma/resolve/main/chroma-unlocked-v48.safetensors\",\n                         downloader.base_path / \"models/unet/Chroma\"))\n    if chroma_v48_detail.value:\n        downloads.append((\"https://huggingface.co/lodestones/Chroma/resolve/main/chroma-unlocked-v48-detail-calibrated.safetensors\",\n                         downloader.base_path / \"models/unet/Chroma\"))\n    if chroma_v29.value:\n        downloads.append((\"https://huggingface.co/lodestones/Chroma/resolve/main/chroma-unlocked-v29.5.safetensors\",\n                         downloader.base_path / \"models/unet/Chroma\"))\n    \n    # Always download VAE and TE if any Chroma model selected\n    if any([chroma_v48.value, chroma_v48_detail.value, chroma_v29.value]):\n        downloads.append((\"https://huggingface.co/lodestones/Chroma/resolve/main/ae.safetensors\",\n                         downloader.base_path / \"models/vae/Chroma\"))\n        if chroma_te.value:\n            downloads.append((\"https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn_scaled.safetensors\",\n                             downloader.base_path / \"models/text_encoders/Chroma\"))\n    \n    # LoRAs\n    lora_path = downloader.base_path / \"models/loras/Chroma\"\n    if lora_realfine.value:\n        downloads.append((\"https://huggingface.co/silveroxides/Chroma-LoRA-Experiments/resolve/main/Chroma-RealFine_lora_rank_64-bf16.safetensors\", lora_path))\n    if lora_turbo.value:\n        downloads.append((\"https://huggingface.co/silveroxides/Chroma-LoRA-Experiments/resolve/main/Chroma-Turbo_lora_rank_64-bf16.safetensors\", lora_path))\n    if lora_hyper8.value:\n        downloads.append((\"https://huggingface.co/silveroxides/Chroma-LoRA-Experiments/resolve/main/Hyper-CHROMA-8steps-lora-minimal.safetensors\", lora_path))\n    if lora_hyper16.value:\n        downloads.append((\"https://huggingface.co/silveroxides/Chroma-LoRA-Experiments/resolve/main/Hyper-Chroma-Turbo-Alpha-16steps-lora.safetensors\", lora_path))\n    \n    # FLUX\n    if flux_schnell.value:\n        downloads.append((\"https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/flux1-schnell.safetensors\",\n                         downloader.base_path / \"models/unet\"))\n    if flux_dev.value:\n        downloads.append((\"https://huggingface.co/black-forest-labs/FLUX.1-dev/resolve/main/flux1-dev.safetensors\",\n                         downloader.base_path / \"models/unet\"))\n    \n    # SDXL\n    if sdxl_base.value:\n        downloads.append((\"https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors\",\n                         downloader.base_path / \"models/checkpoints\"))\n    if sdxl_refiner.value:\n        downloads.append((\"https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors\",\n                         downloader.base_path / \"models/checkpoints\"))\n    if sdxl_vae.value:\n        downloads.append((\"https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors\",\n                         downloader.base_path / \"models/vae\"))\n    \n    # Download all\n    if downloads:\n        print(f\"\\n🚀 Starting download of {len(downloads)} models to {downloader.base_path}\\n\")\n        for url, path in downloads:\n            downloader.download_file(url, path)\n        print(\"\\n✅ All downloads completed!\")\n        \n        # Create symlinks\n        if downloader.base_path != PROJECT_PATH:\n            print(\"\\n🔗 Creating symlinks to ComfyUI...\")\n            for model_dir in ['models/unet', 'models/vae', 'models/loras', 'models/text_encoders', 'models/checkpoints']:\n                source = downloader.base_path / model_dir\n                target = PROJECT_PATH / model_dir\n                if source.exists() and not target.exists():\n                    target.parent.mkdir(parents=True, exist_ok=True)\n                    os.symlink(source, target)\n                    print(f\"   ✅ Linked: {model_dir}\")\n    else:\n        print(\"❌ No models selected!\")\n\ndownload_btn = widgets.Button(description='Download Selected', button_style='success', icon='download')\ndownload_btn.on_click(download_selected)\n\ndisplay(download_btn)\ndisplay(downloader.progress)\ndisplay(downloader.status)

## 🗑️ 3. Storage Cleanup

In [ ]:
# Cleanup temporary storage\ndef cleanup_temp_storage():\n    if TEMP_STORAGE.exists():\n        size = sum(f.stat().st_size for f in TEMP_STORAGE.rglob('*') if f.is_file()) / (1024**3)\n        print(f\"Temporary storage size: {size:.2f}GB\")\n        \n        confirm = input(\"Delete temporary storage? (yes/no): \")\n        if confirm.lower() == 'yes':\n            shutil.rmtree(TEMP_STORAGE)\n            print(\"✅ Temporary storage cleared\")\n        else:\n            print(\"❌ Cleanup cancelled\")\n    else:\n        print(\"No temporary storage to clean\")\n\n# Uncomment to run:\n# cleanup_temp_storage()

## 📝 4. List Downloaded Models

In [ ]:
# List all downloaded models\ndef list_models():\n    print(\"📦 DOWNLOADED MODELS\")\n    print(\"==================\")\n    \n    for storage_name, storage_path in [(\"Permanent\", STORAGE_PATH), (\"Temporary\", TEMP_STORAGE), (\"Project\", PROJECT_PATH)]:\n        if storage_path.exists():\n            models = list(storage_path.rglob('*.safetensors')) + list(storage_path.rglob('*.gguf'))\n            if models:\n                print(f\"\\n📁 {storage_name} Storage:\")\n                for model in models:\n                    size_mb = model.stat().st_size / (1024**2)\n                    rel_path = model.relative_to(storage_path)\n                    print(f\"   • {rel_path} ({size_mb:.1f}MB)\")\n\nlist_models()

---\n\n## 🎉 **Model Management Complete!**\n\n**Your models are ready to use in ComfyUI**\n\n### 💡 **Tips:**\n\n- **Temporary Storage**: Fast but lost on restart\n- **Permanent Storage**: Limited to 50GB total\n- **Symlinks**: Automatically created to ComfyUI\n\n---